In [1]:
from near_functions import query_near_testnet
import sys
import subprocess
import re

In [2]:
q = """SELECT emitted_by_contract_account_id AS contract, token_id AS token_id
FROM public.assets__non_fungible_token_events
WHERE token_old_owner_account_id in ('') AND event_kind in ('MINT')
ORDER BY emitted_at_block_timestamp DESC 
LIMIT 10;;"""

results = query_near_testnet(q)

## Syntax for reference

Work on methods to programatically input new CLI commands with this `subprocess.run` function

`subprocess.run(["near","view","\"nft.lamns3.testnet\"","nft_token","{\"token_id\":\"415\"}"],capture_output=True, text=True).stdout.strip('\n')` 


### Change index to view below with `index_to_query`

In [3]:
index_to_query = 3

In [4]:
contract = results[index_to_query]['emitted_by_contract_account_id']
token = results[index_to_query]['token_id']
contract, token

('nft.minimous36.testnet', '2:2')

In [5]:
contract = r'\"{}"'.format(contract)

In [6]:
token = """{{\"token_id\":\"{}\"}}""".format(token)

In [7]:
contract, token

('\\"nft.minimous36.testnet"', '{"token_id":"2:2"}')

In [8]:
# TODO - Slicing the contract variable is necessary to get results in some queries,
# but it sometimes is unneeded and will crop off the first letter of the contract name
# Try changing index_to_query in cell 3. 

x = subprocess.run(
    [
        "near","view", contract[1:], "nft_token", token
    ],
    capture_output=True, text=True
).stdout.strip('\n').lstrip("View call: ")

### Results:

In [9]:
x

'nft.minimous36.testnet.nft_token({"token_id":"2:2"})\n{\n  token_id: \'2:2\',\n  owner_id: \'minimous36.testnet\',\n  metadata: {\n    title: \'Test #1 #2\',\n    description: null,\n    media: \'bafybeidzcan4nzcz7sczs4yzyxly4galgygnbjewipj6haco4kffoqpkiy\',\n    media_hash: null,\n    copies: 10,\n    issued_at: \'1659561672736435854\',\n    expires_at: null,\n    starts_at: null,\n    updated_at: null,\n    extra: null,\n    reference: \'bafybeicg4ss7qh5odijfn2eogizuxkrdh3zlv4eftcmgnljwu7dm64uwji\',\n    reference_hash: null\n  },\n  approved_account_ids: {}\n}'

## Data Cleaning Below

In [10]:
new = x.split("\n")

In [11]:
new = [i.strip() for i in new]
new

['nft.minimous36.testnet.nft_token({"token_id":"2:2"})',
 '{',
 "token_id: '2:2',",
 "owner_id: 'minimous36.testnet',",
 'metadata: {',
 "title: 'Test #1 #2',",
 'description: null,',
 "media: 'bafybeidzcan4nzcz7sczs4yzyxly4galgygnbjewipj6haco4kffoqpkiy',",
 'media_hash: null,',
 'copies: 10,',
 "issued_at: '1659561672736435854',",
 'expires_at: null,',
 'starts_at: null,',
 'updated_at: null,',
 'extra: null,',
 "reference: 'bafybeicg4ss7qh5odijfn2eogizuxkrdh3zlv4eftcmgnljwu7dm64uwji',",
 'reference_hash: null',
 '},',
 'approved_account_ids: {}',
 '}']

In [12]:
type(new[7])

str

In [13]:
found_media = ''

for item in new:
    if 'View call' in item:
        new.remove(item)
        continue
    if 'media:' in item:
        found_media = item
    if len(item) < 5:
        new.remove(item)
        
print(new)
print(found_media)

if len(found_media) > 1 and "null" not in found_media:
    found_media = found_media.lstrip("media: '")
    found_media = found_media.rstrip("',")
    print()
    print(found_media)

['nft.minimous36.testnet.nft_token({"token_id":"2:2"})', "token_id: '2:2',", "owner_id: 'minimous36.testnet',", 'metadata: {', "title: 'Test #1 #2',", 'description: null,', "media: 'bafybeidzcan4nzcz7sczs4yzyxly4galgygnbjewipj6haco4kffoqpkiy',", 'media_hash: null,', 'copies: 10,', "issued_at: '1659561672736435854',", 'expires_at: null,', 'starts_at: null,', 'updated_at: null,', 'extra: null,', "reference: 'bafybeicg4ss7qh5odijfn2eogizuxkrdh3zlv4eftcmgnljwu7dm64uwji',", 'reference_hash: null', 'approved_account_ids: {}']
media: 'bafybeidzcan4nzcz7sczs4yzyxly4galgygnbjewipj6haco4kffoqpkiy',

bafybeidzcan4nzcz7sczs4yzyxly4galgygnbjewipj6haco4kffoqpkiy


## FIN